In [1]:
import pandas as pd
# Import block
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [2]:
def nbetageomLL(par, data):
    a = par[0]
    b = par[1]
    ldenom = math.lgamma(a)+math.lgamma(b)-math.lgamma(
        a+b)
    llsum = 0
    for t in range(1,len(data)):
        lnumer = math.lgamma(a+1)+math.lgamma(b+t-1)-math.lgamma(a+b+t)
        llsum += (data[t-1]-data[t])*(lnumer-ldenom)
    lnumer_surv = math.lgamma(a)+math.lgamma(b+len(data)-1)-math.lgamma(a+b+len(data)-1)
    llsum += data[len(data)-1]*(lnumer_surv - ldenom)
    return -llsum



# Code provided by Dr. Arun Gopalakrishnan to optimize the theta by 
# solving for a and b in the best way. Returns res, which is a list
# of two elements, res[0] = a, res[1] = b
def optimize(data):
    bnds = ((0.0001,None),(0.0001,None)) 
    res = minimize(nbetageomLL, (1,1), (data), method="nelder-mead",
                   options={'xatol': 1e-08, 'disp': True},bounds=bnds)
    print('MLE for a,b is', res.x, 'and LL is',-nbetageomLL(res.x,data))
    return res

In [3]:
def cohort_valuation(cohort_list, years_after_acquisition, discount, v_bar):
    
    #Since RLV beginning at t>0 is equivalent to the CLV, we'll simply
    #return the CLV of the given cohort_list's first element
    if years_after_acquisition == 0:
        
        #Initialize a list of percent of customers active in which the 
        #first element contains 100% of active customers 
        percent_list = [cohort_list[0]]
        
        #Next, we'll get a list of the percentage of the active population
        #in subsequent time periods by dividing the nth element by the 
        #number we started with
        percent_list = [population/percent_list[0] for population in cohort_list]
        
        #In addition, we need to make a list of the future of the 
        #discount rate
        disc_future = []
        
        #The formula for the discount rate. This will be raised to the 
        #nth power to compute our future discount and appended to the
        #list
        formula = 1/(1 + discount)
        
        #Since we only care about the CLV in this case, we'll initialize
        #it to 0 and make it a rolling total
        clv = 0
        
        
        for i in range(len(cohort_list)):
            #Appending the future discount to our list as discussed
            disc_future.append(formula**i)
            
            #Based on the value of i, we'll multiply the ith element
            #of the input cohort_list by its respective percent_list's
            #ith element and divide by our discount at that time.
            clv += (cohort_list[i] * percent_list[i])/(formula**i)
            
        #When the for loop is done, we have our CLV and we return it
        return clv
    
    #When years after acquisition is greater than 0, we need to compute
    #our cohort_list's a and b. We do this by calling the optimize method
    #which returns an object where x is a list of two elements, a and b 
    #respectively
    res = optimize(cohort_list)
    a = res.x[0]
    b = res.x[1]
    
    #Time is an arbitrary list going up to 200 in this case. In every case
    #but the instance where a person includes a membership in their will,
    #the customer's lifetime value will greatly be under 200, but it 
    #suffices for the purposes of convergence we're looking for in our
    #program
    time = []
    time.extend(range(1,201))
    
    #Retention will be computed, but at year 0 you haven't retained any 
    #customers because they are only being acquired. 
    retention = [None]
    
    #For every element in time, we will compute a retention rate using
    #the formula (b + time - 1) / (a + b + time - 1). This will be our
    #program's r(t) column
    for i in time:
        retention.append((b + i - 1) / (a + b + i - 1))
        
    #Once we have our retention list, we can use the input amount of 
    #years after acquisition to slice the retention list beginning
    #at the year we need to start with
    rlv_future = retention[years_after_acquisition:]
    
    #The first step is assigning our retention rate at the given 
    #time period to the first element of a new list called 
    #surv_future. This is our program's S(t) column
    surv_future = [rlv_future[0]]
    
    #From there, we'll be looking at rlv_future's values and 
    #multiplying the previous time period's S(t) and multiplying
    #it by the current r(t) to produce the current time period's
    #S(t). Once we get that number, we append it to the
    #surv_future list
    for i in range(1, len(rlv_future)):
        surv_future.append(surv_future[i-1] * rlv_future[i])
        #print(rlv_future[i], surv_future[i-1])

    #In addition, we need to make a list of the future of the 
    #discount rate
    disc_future = []
            
    #The formula for the discount rate. This will be raised to the 
    #nth power to compute our future discount and appended to the
    #list
    formula = 1/(1 + discount)
    
    #Sumproduct is a function you can use in Excel, but all it does
    #is compute the sum of a list of products so we'll mimic that 
    #behavior in Python. We begin with 0 and use it as a rolling sum
    sumproduct = 0

    #We populate the disc_future list and immediately after appending
    #the nth element of the disc_future list, we multiply the nth
    #elements of surv_future and disc_future before adding the product
    #to sumproduct
    for i in range(len(surv_future)):
        disc_future.append(formula**i)
        sumproduct += surv_future[i] * disc_future[i] 
        
    #Now that we have the sumproduct, we can multiply it by the
    #input v_bar value and return the sumproduct 
    sumproduct *= v_bar
    
    return sumproduct

In [4]:
#The monthly additions to the customer base provided in the file
adds = [500, 500, 1000, 1000, 1000, 1000]

#The monthly additions to the customer base provided in the file
losses = [135, 215, 405, 525, 605]

#List of cumulative totals. Will be populated later in the program
cumu_totals = []

#Losses within the cohort of 500 and 1000. We'll be appending to 
#these lists later in the program as we compute the losses of
#each cohort within the given time period
losses_500 = [135]
losses_1000 = [270]

#When these are complete, we'll have the behavior of the cohort up 
#to t elements, where t is time we're observing the cohort
cohort_track_500 = [500]
cohort_track_1000 = [1000]

In [5]:
'''
I'm positive I can make this into a loop given some more time, but
for now it's a linear iteration through a given number of 
iterations, in this case 6. The behaviors of each iteration 
minus i=0 are mostly similar with the difference being num
and the amount of elements it's using to compute it. There's
also the inconsistency in appending the element to  
'''

#Iteration 0
i = 0

#We set our "instance variable" to the first element of the 
#global losses list
num = losses[i] 

#Since it's the first iteration, we don't need to do any
#further computation and can just print the variable
print("Number of customers lost from cohort 1:", num)

#We now append the number of active users minus the loss
#from the cohort to the track of active users and multiply
#by 2 before appending to 1000, which we're assuming will
#behave the same way
cohort_track_500.append(cohort_track_500[i] - losses_500[i])
cohort_track_1000.append(2 * (cohort_track_500[i] - losses_500[i]))

#Printing results to screen to illustrate steps, uncomment
#to see results
print("cohort tracks")
print(cohort_track_500)
print(cohort_track_1000)
print()
print("losses")
print(losses_500)
print(losses_1000)
print()

#Iteration 1
i = 1

#From this point on, the computation is different. We 
#would like for num to represent the number of customers
#lost from the first cohort, so to do that we take the 
#total number of losses at time t=i (t=1 in this case)
#and subtract the number of known losses to arrive at 
#the number of losses from our original cohort. For
#example, in this case num will be set to 215-135, 80
num = losses[i] - losses_500[i-1]
print("Number of customers lost from cohort 1:", num)

#Since we're arriving at new information, we need to 
#record it by adding the number 80 to the list of 
#sequential losses from the original cohort. We also
#multiply it by 2 and 
losses_500.append(num)
losses_1000.append(2 * num)

#We now append the number of active users minus the loss
#from the cohort to the track of active users and multiply
#by 2 before appending to 1000, which we're assuming will
#behave the same way

cohort_track_500.append(cohort_track_500[i] - num)
cohort_track_1000.append((cohort_track_500[-1]) * 2)

#Printing results to screen to illustrate steps
print("cohort tracks")
print(cohort_track_500)
print(cohort_track_1000)
print()
print("losses")
print(losses_500)
print(losses_1000)
print()

#Iteration 2
i = 2

#num = 405 - 270 - 80 = 55
#no other changes
num = losses[i] - losses_1000[0] - losses_500[1]
print("Number of customers lost from cohort 1:", num)
losses_500.append(num)
losses_1000.append(2 * num)
cohort_track_500.append(cohort_track_500[-1] - num)
cohort_track_1000.append(cohort_track_500[-1] * 2)

#Printing results to screen to illustrate steps
print("cohort tracks")
print(cohort_track_500)
print(cohort_track_1000)
print()
print("losses")
print(losses_500)
print(losses_1000)
print()

#Iteration 3
i = 3

#num = 525 - 270 - 160 - 55 = 40
num = losses[i] - losses_1000[0] - losses_1000[1]- losses_500[2]
print("Number of customers lost from cohort 1:", num)
losses_500.append(num)
losses_1000.append(2 * num)
cohort_track_500.append(cohort_track_500[-1] - num)
cohort_track_1000.append(cohort_track_500[-1] * 2)

#Printing results to screen to illustrate steps
print("cohort tracks")
print(cohort_track_500)
print(cohort_track_1000)
print()
print("losses")
print(losses_500)
print(losses_1000)
print()

#No appends to the losses because we've filled the table after this is done
i = 4

#num = 605 - 270 - 160 - 110 - 40 = 25
num = losses[i] - losses_1000[0] - losses_1000[1] - losses_1000[2]- losses_500[3]
print("Number of customers lost from cohort 1:", num)
cohort_track_500.append(cohort_track_500[-1] - num)
cohort_track_1000.append(cohort_track_500[-1] * 2)

#Printing results to screen to illustrate steps
print("cohort tracks")
print(cohort_track_500)
print(cohort_track_1000)
print()
print("losses")
print(losses_500)
print(losses_1000)
print()

Number of customers lost from cohort 1: 135
cohort tracks
[500, 365]
[1000, 730]

losses
[135]
[270]

Number of customers lost from cohort 1: 80
cohort tracks
[500, 365, 285]
[1000, 730, 570]

losses
[135, 80]
[270, 160]

Number of customers lost from cohort 1: 55
cohort tracks
[500, 365, 285, 230]
[1000, 730, 570, 460]

losses
[135, 80, 55]
[270, 160, 110]

Number of customers lost from cohort 1: 40
cohort tracks
[500, 365, 285, 230, 190]
[1000, 730, 570, 460, 380]

losses
[135, 80, 55, 40]
[270, 160, 110, 80]

Number of customers lost from cohort 1: 25
cohort tracks
[500, 365, 285, 230, 190, 165]
[1000, 730, 570, 460, 380, 330]

losses
[135, 80, 55, 40]
[270, 160, 110, 80]



In [6]:
#Because our list of losses is one element shorter
#than our adds list, we'll insert a 0 at element 0 
#to line the lists up for this process. It'll 
#effectively mean that in the month we acquired 500
#customers, none of them churned
losses.insert(0, 0)

#Loop that will give us the cumulative totals of
#active customers at time t
for i in range(len(adds)):
    num = i
    total = 0
    while num >= 0:
        total += adds[num]
        total -= losses[num]
        num -=1
    cumu_totals.append(total)
cumu_totals

[500, 865, 1650, 2245, 2720, 3115]

In [7]:
#Because we have to compute a different RLV for each cohort,
#we'll store each RLV in a list of RLVs and then get the sum 
#of those elements later. Now that we have the cohort's 
#population track, we can send it in with the number of 
#years after acquisition we're wanting to start RLV calculations
#at, along with the discount and the v_bar amount
 
rlv_list = []

#[starting 5 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_500, 5, .1, 500))

#[starting 4 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_500, 4, .1, 500))

#[starting 3 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_1000, 3, .1, 500))

#[starting 2 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_1000, 2, .1, 500))

#[starting 1 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_1000, 1, .1, 500))

#[starting 0 years after acquisition, #10% discount, v_bar = 500
rlv_list.append(cohort_valuation(cohort_track_1000, 0, .1, 500))

Optimization terminated successfully.
         Current function value: 803.867761
         Iterations: 73
         Function evaluations: 152
MLE for a,b is [1.24547838 3.36361985] and LL is -803.867760634909
Optimization terminated successfully.
         Current function value: 803.867761
         Iterations: 73
         Function evaluations: 152
MLE for a,b is [1.24547838 3.36361985] and LL is -803.867760634909
Optimization terminated successfully.
         Current function value: 1607.735521
         Iterations: 73
         Function evaluations: 152
MLE for a,b is [1.24547838 3.36361985] and LL is -1607.735521269818
Optimization terminated successfully.
         Current function value: 1607.735521
         Iterations: 73
         Function evaluations: 152
MLE for a,b is [1.24547838 3.36361985] and LL is -1607.735521269818
Optimization terminated successfully.
         Current function value: 1607.735521
         Iterations: 73
         Function evaluations: 152
MLE for a,b is [1.2454

/Users/tyleraguilar/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:535: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  warn('Method %s cannot handle constraints nor bounds.' % method,


In [8]:
#A couple housekeeping lines that I only want to run once.
#They used to be on the following block but make it hard 
#to troubleshoot
cumu_totals.append(0)
rlv_list.append(sum(rlv_list))

In [9]:
#Using the pandas library, we can make our visualization pretty clear
df = pd.DataFrame(np.empty, index=[1, 2, 3, 4, 5, 6, "Totals"],
                  columns=['1', '2', '3', '4', '5', '6', 'Cohort RLV'])

#We'll use both lists here in populating the table 
t500 = cohort_track_500
t1000 = cohort_track_1000
df.loc[1] = pd.Series({'1':t500[0], '2':t500[1], '3':t500[2], '4':t500[3], '5':t500[4], '6':int(t500[5])})
df.loc[2] = pd.Series({'2':int(t500[0]), '3':int(t500[1]), '4':int(t500[2]), '5':int(t500[3]), '6':int(t500[4])})
df.loc[3] = pd.Series({'3':t1000[0], '4':t1000[1], '5':t1000[2], '6':t1000[3]})
df.loc[4] = pd.Series({'4':t1000[0], '5':t1000[1], '6':t1000[2]})
df.loc[5] = pd.Series({'5':t1000[0], '6':t1000[1]})
df.loc[6] = pd.Series({'6':t1000[0]})

#Cumulative totals will be displayed in the bottom row called Totals
df.loc["Totals"] = pd.Series({'1':cumu_totals[0], '2':cumu_totals[1], '3':cumu_totals[2], 
                              '4':cumu_totals[3], '5':cumu_totals[4], '6':cumu_totals[5]})

#Formatting the numbers in RLV to appear as currency and then
#filling a column called Cohort RLV with money data
rlv_list = ['${:0,.0f}'.format(rlv) for rlv in rlv_list]
df['Cohort RLV'] = rlv_list

#Housekeeping here to make our table more intuitive by removing 
#NaN and 0 values. We also had an issue with numbers appearing 
#as floats, so we make sure to make them integers before presenting
df = df.fillna(0)
for i in range(1,7):    
    df[str(i)] = df[str(i)].astype('int')
df = df.replace(0, '')

In [10]:
#Run to view the dataframe
df

,1,2,3,4,5,6,Cohort RLV
1,500,365,285,230,190,165,"$2,434"
2,,500,365,285,230,190,"$2,269"
3,,,1000,730,570,460,"$2,080"
4,,,,1000,730,570,"$1,860"
5,,,,,1000,730,"$1,599"
6,,,,,,1000,"$2,648"
Totals,500,865,1650,2245,2720,3115,"$12,890"


In [11]:
print("The Residual Lifetime Value of the", df['6']['Totals'], 
      "customers is:", df['Cohort RLV']['Totals'])

The Residual Lifetime Value of the 3115 customers is: $12,890
